In [1]:
import pandas as pd
import snowflake.connector

In [2]:
# connect to Snowflake Account
conn = snowflake.connector.connect(
    user="USERNAME",
    password="Your_Sn0wflak3_Password",
    account="IMTFQZV-DPC01467",
    warehouse="COMPUTE_WH")

In [ ]:
#Define block range parameters within SQL query
cur = conn.cursor()
cur.execute("SET min_block_id = 379995000")
cur.execute("SET max_block_id = 380000000")
cur.execute("SET max_limit   =  10000000000")
cur.execute("SET min_season_block = 379995000")
cur.execute("SET max_season_block = 380000000")

### Note:
The above block range is only for fast exectution. For our final results, we used the block ranges below, which we exported as csv and load into pandas later in the code. The goal was to enable this notebook to be run by anyone without incurring high compute costs or long wait times, yet still provide the full data for analysis.
<br>
<br>
<br>
What we actually used for the final query:<br>
cur.execute("SET min_block_id = 379899000") <br>
cur.execute("SET max_block_id = 380000000")<br>
cur.execute("SET max_limit   =  100000000000")<br>
cur.execute("SET min_season_block = 300000000")<br>
cur.execute("SET max_season_block = 380000000")


In [4]:
main = """

--Define all the intermediary dataframes
WITH

/*
Define t1 as the "main" dataset we will JOIN on later.
This is the dataset containing all the fee columns.
We have to flatten instructions twice in order to access the account value
This account value corresponds to the ORCA SOL-USDC pool
Moreover, we rank over priority_fee to get the ordering within the same block, both in absolute and quantile values.
We take this data from the SOLANA_ONCHAIN_CORE_DATA.CORE.FACT_TRANSACTIONS database, containing all transaction details (importantly fees)
*/

--Define t1 as the main one, with priority fees
t1 AS (
    SELECT *
    FROM (
        SELECT
            BLOCK_ID AS block,
            FEE AS fee,
            TX_ID AS tx_id,
            FEE - 5000 AS priority_fee,

            RANK() OVER (
                PARTITION BY BLOCK_ID
                ORDER BY (FEE - 5000) DESC
            ) AS fee_rank_desc,

            (RANK() OVER (
                PARTITION BY BLOCK_ID
                ORDER BY (FEE - 5000) DESC
            ) * 1.0)
            /
            (COUNT(*) OVER (PARTITION BY BLOCK_ID)) AS quantile_position

        FROM SOLANA_ONCHAIN_CORE_DATA.CORE.FACT_TRANSACTIONS AS tr,
             LATERAL FLATTEN(input => tr.instructions) AS inst,
             LATERAL FLATTEN(input => inst.value:accounts) AS acct

        WHERE
            BLOCK_ID > $min_block_id
            AND BLOCK_ID < $max_block_id
            AND SUCCEEDED = TRUE
            AND acct.value::string = 'FksffEqnBRixYGR791Qw2MgdU7zNCpHVFYBL4Fa4qVuH'

        ORDER BY BLOCK_ID, TX_ID
        LIMIT $max_limit
    )
),

/*
We now create multiple datasets, each for the price of a specific pool
Conceptually, we take the dataset containing the account changes, select the USDC token (''EPjF...),
compute the absolute difference between before and after the transaction. We do the same for the SOL token ('So111...')
Then, We divide the absolute change in USDC by the absolute change in SOL and take it as the price.
We do this because it is not possible to get a reliable data for price per block per account. Moreover, this seems to be
surprisingly accurate, as most accounts have very similar prices and it matches the real, historic price we can see on M1 candles
on charting softwares.
Finally, we only take rows where the owner (account) is the pool that interests us.


We had to take this data from the SOLANA_ONCHAIN_CORE_DATA.CORE.FACT_TOKEN_BALANCES database, containg information on token balances (which
we use to reconstruct price)
*/


t2_Fksff AS (
    SELECT *
    FROM (
        SELECT
            MIN(BLOCK_ID) AS BLOCK_ID,
            MIN(TX_INDEX) AS TX_INDEX,

            ABS(
                SUM(
                    CASE
                        WHEN MINT = 'EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v'
                        THEN BALANCE - PRE_BALANCE
                        ELSE 0
                    END
                )
            )
            /
            NULLIF(
                ABS(
                    SUM(
                        CASE
                            WHEN MINT = 'So11111111111111111111111111111111111111112'
                            THEN BALANCE - PRE_BALANCE
                            ELSE 0
                        END
                    )
                ),
                0
            ) AS Fksff_price

        FROM SOLANA_ONCHAIN_CORE_DATA.CORE.FACT_TOKEN_BALANCES
        WHERE
            BLOCK_ID > $min_block_id
            AND BLOCK_ID < $max_block_id
            AND OWNER = 'FksffEqnBRixYGR791Qw2MgdU7zNCpHVFYBL4Fa4qVuH'

        GROUP BY TX_ID
        ORDER BY BLOCK_ID, TX_ID
        LIMIT $max_limit
    )
),

t2_Czfq3 AS (
    SELECT *
    FROM (
        SELECT
            MIN(BLOCK_ID) AS BLOCK_ID,
            MIN(TX_INDEX) AS TX_INDEX,

            ABS(
                SUM(
                    CASE
                        WHEN MINT = 'EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v'
                        THEN BALANCE - PRE_BALANCE
                        ELSE 0
                    END
                )
            )
            /
            NULLIF(
                ABS(
                    SUM(
                        CASE
                            WHEN MINT = 'So11111111111111111111111111111111111111112'
                            THEN BALANCE - PRE_BALANCE
                            ELSE 0
                        END
                    )
                ),
                0
            ) AS Czfq3_price

        FROM SOLANA_ONCHAIN_CORE_DATA.CORE.FACT_TOKEN_BALANCES
        WHERE
            BLOCK_ID > $min_block_id
            AND BLOCK_ID < $max_block_id
            AND OWNER = 'Czfq3xZZDmsdGdUyrNLtRhGc47cXcZtLG4crryfu44zE'

        GROUP BY TX_ID
        ORDER BY BLOCK_ID, TX_ID
        LIMIT $max_limit
    )
),

t2_DB3sU AS (
    SELECT *
    FROM (
        SELECT
            MIN(BLOCK_ID) AS BLOCK_ID,
            MIN(TX_INDEX) AS TX_INDEX,

            ABS(
                SUM(
                    CASE
                        WHEN MINT = 'EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v'
                        THEN BALANCE - PRE_BALANCE
                        ELSE 0
                    END
                )
            )
            /
            NULLIF(
                ABS(
                    SUM(
                        CASE
                            WHEN MINT = 'So11111111111111111111111111111111111111112'
                            THEN BALANCE - PRE_BALANCE
                            ELSE 0
                        END
                    )
                ),
                0
            ) AS DB3sU_price

        FROM SOLANA_ONCHAIN_CORE_DATA.CORE.FACT_TOKEN_BALANCES
        WHERE
            BLOCK_ID > $min_block_id
            AND BLOCK_ID < $max_block_id
            AND OWNER = 'DB3sUCP2H4icbeKmK6yb6nUxU5ogbcRHtGuq7W2RoRwW'

        GROUP BY TX_ID
        ORDER BY BLOCK_ID, TX_ID
        LIMIT $max_limit
    )
),

t2_n9VhC AS (
    SELECT *
    FROM (
        SELECT
            MIN(BLOCK_ID) AS BLOCK_ID,
            MIN(TX_INDEX) AS TX_INDEX,

            ABS(
                SUM(
                    CASE
                        WHEN MINT = 'EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v'
                        THEN BALANCE - PRE_BALANCE
                        ELSE 0
                    END
                )
            )
            /
            NULLIF(
                ABS(
                    SUM(
                        CASE
                            WHEN MINT = 'So11111111111111111111111111111111111111112'
                            THEN BALANCE - PRE_BALANCE
                            ELSE 0
                        END
                    )
                ),
                0
            ) AS n9VhC_price

        FROM SOLANA_ONCHAIN_CORE_DATA.CORE.FACT_TOKEN_BALANCES
        WHERE
            BLOCK_ID > $min_block_id
            AND BLOCK_ID < $max_block_id
            AND OWNER = '6n9VhCwQ7EwK6NqFDjnHPzEk6wZdRBTfh43RFgHQWHuQ'

        GROUP BY TX_ID
        ORDER BY BLOCK_ID, TX_ID
        LIMIT $max_limit
    )
),

t2_AvGeF AS (
    SELECT *
    FROM (
        SELECT
            MIN(BLOCK_ID) AS BLOCK_ID,
            MIN(TX_INDEX) AS TX_INDEX,

            ABS(
                SUM(
                    CASE
                        WHEN MINT = 'EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v'
                        THEN BALANCE - PRE_BALANCE
                        ELSE 0
                    END
                )
            )
            /
            NULLIF(
                ABS(
                    SUM(
                        CASE
                            WHEN MINT = 'So11111111111111111111111111111111111111112'
                            THEN BALANCE - PRE_BALANCE
                            ELSE 0
                        END
                    )
                ),
                0
            ) AS AvGeF_price

        FROM SOLANA_ONCHAIN_CORE_DATA.CORE.FACT_TOKEN_BALANCES
        WHERE
            BLOCK_ID > $min_block_id
            AND BLOCK_ID < $max_block_id
            AND OWNER = 'AvGeFw71N5sNfV97mZ1uNrHg4yfufRicCJUrS9j2ehTX'

        GROUP BY TX_ID
        ORDER BY BLOCK_ID, TX_ID
        LIMIT $max_limit
    )
),

t2_ucNos AS (
    SELECT *
    FROM (
        SELECT
            MIN(BLOCK_ID) AS BLOCK_ID,
            MIN(TX_INDEX) AS TX_INDEX,

            ABS(
                SUM(
                    CASE
                        WHEN MINT = 'EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v'
                        THEN BALANCE - PRE_BALANCE
                        ELSE 0
                    END
                )
            )
            /
            NULLIF(
                ABS(
                    SUM(
                        CASE
                            WHEN MINT = 'So11111111111111111111111111111111111111112'
                            THEN BALANCE - PRE_BALANCE
                            ELSE 0
                        END
                    )
                ),
                0
            ) AS ucNos_price

        FROM SOLANA_ONCHAIN_CORE_DATA.CORE.FACT_TOKEN_BALANCES
        WHERE
            BLOCK_ID > $min_block_id
            AND BLOCK_ID < $max_block_id
            AND OWNER = '3ucNos4NbumPLZNWztqGHNFFgkHeRMBQAVemeeomsUxv'

        GROUP BY TX_ID
        ORDER BY BLOCK_ID, TX_ID
        LIMIT $max_limit
    )
),

/*
In this dataset, we compute the total volume accross multiple accounts. We apply the same trick as before,
summing the absolute values of the difference between pre and post balances (only USDC this time).
We group by block id, since we want volume for the whole block
*/


t3 AS (
    SELECT
        MIN(BLOCK_ID) AS BLOCK_ID,
        AVG(
            CASE
                WHEN OWNER = 'Czfq3xZZDmsdGdUyrNLtRhGc47cXcZtLG4crryfu44zE'
                     AND MINT = 'EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v'
                THEN BALANCE
            END
        ) AS avg_balance,

        SUM(
            CASE
                WHEN MINT IN (
                    'EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v',
                    'Es9vMFrzaCERmJfrF4H2FYD4KCoNkY11McCe8BenwNYB'
                )
                THEN ABS(BALANCE - PRE_BALANCE)
                ELSE 0
            END
        ) AS volume

    FROM SOLANA_ONCHAIN_CORE_DATA.CORE.FACT_TOKEN_BALANCES
    WHERE
        BLOCK_ID > $min_block_id
        AND BLOCK_ID < $max_block_id
        AND OWNER IN (
            'FksffEqnBRixYGR791Qw2MgdU7zNCpHVFYBL4Fa4qVuH',
            'Czfq3xZZDmsdGdUyrNLtRhGc47cXcZtLG4crryfu44zE',
            'DB3sUCP2H4icbeKmK6yb6nUxU5ogbcRHtGuq7W2RoRwW',
            '6n9VhCwQ7EwK6NqFDjnHPzEk6wZdRBTfh43RFgHQWHuQ',
            'AvGeFw71N5sNfV97mZ1uNrHg4yfufRicCJUrS9j2ehTX',
            '3ucNos4NbumPLZNWztqGHNFFgkHeRMBQAVemeeomsUxv',
            '58oQChx4yWmvKdwLLZzBi4ChoCc2fqCUWBkwMihLYQo2',
            '7VuKeevbvbQQcxz6N4SNLmuq6PYy4AcGQRDssoqo4t65',
            'FLckHLGMJy5gEoXWwcE68Nprde1D4araK4TGLw4pQq2n',
            'BGm1tav58oGcsQJehL9WXBFXF7D27vZsKefj4xJKD5Y',
            '8Pm2kZpnxD3hoMmt4bjStX2Pw2Z9abpbHzZxMPqxPmie',
            'FwewVm8u6tFPGewAyHmWAqad9hmF7mvqxK4mJ7iNqqGC',
            '3nMFwZXwY1s1M5s8vYAHqd4wGs4iSxXE4LRoUMMYqEgF',
            'EiEAydLqSKFqRPpuwYoVxEJ6h9UZh9tsTaHgs4f8b8Z5',
            '7XawhbbxtsRcQA8KTkHT9f9nc6d69UwqCDh6U5EEbEmX'
        )

    GROUP BY BLOCK_ID
    ORDER BY BLOCK_ID
    LIMIT $max_limit
),

/*
We then compute average volume over the last 999 blocks (around 8 minutes), giving a slightly more macro perspective.
We decided to create a new dataset instead of implementing it in t3 directly because we were concerned that the 999 row limit
would only select the last 999 rows, and not the last 999 blocks. (We are slightly unsure if this concern is accurate or not,
but doing it like this seemed more conservative, although less concise)
*/


t3_1 AS (
    SELECT
        BLOCK_ID,
        AVG(volume) OVER (
            ORDER BY BLOCK_ID
            ROWS BETWEEN 999 PRECEDING AND CURRENT ROW
        ) AS avg_volume_last_999_blocks
    FROM t3
),

/*
This dataset sums and averages priority fees paid in each block. We will use this later to compute
the average and summed fees in the last n blocks (creating new columns).
Naturally, we group by block.
*/

t4 AS (
    SELECT
        block,
        SUM(priority_fee) AS block_fee_sum,
        AVG(priority_fee) AS block_fee_avg
    FROM t1
    GROUP BY block
),

/*
Here, we sort of do an intermediary step, joining all the datasets and using this more concentrated dataset for the final database.
This is because we wanted to compute price changes, but this first necessitates some cleaning and doing it in a single SELECT
would have been less understandable.

First, we take all prices from the t2_xxxxx databases and replace with the last recorded price in case the current price is inexistant (possible 
for illiquid pools).
Importantly, this LAST_VALUE function enables us to select only the last transaction of the previous block (found by looking at the highest TX_INDEX).
This ensures that the price we consider is the very last price paid in the previous block.

Then, we add all the columns for the avg and summed priority fees in the previous 10 blocks, defining the delay in the JOIN directly.
Correspondingly, we JOIN most databases on BLOCK_ID + 1, ensuring that the data for the current block come from the last block (which is the last information visible live).
This means for example that the price is the price of the last block, not the current block, effectively avoiding data leakage or useless predictions.

We then do a qualify to keep only one row per tx_id (multiple where created during the prices databases, because one transaction id
is actually split into multiple for each action of the transaction (buy SOL and sell USDC for example)).
*/

pre_final AS (
    SELECT
        t1.block,
        t1.tx_id,
        t1.priority_fee,
        t1.fee_rank_desc,
        t1.quantile_position,
        t3.volume,

        LAST_VALUE(t3.avg_balance IGNORE NULLS) OVER (
            ORDER BY t1.block, t3.block_id
            ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
        ) AS avg_balance,

        LAST_VALUE(t2_Fksff.Fksff_price IGNORE NULLS) OVER (
            ORDER BY t1.block, t2_Fksff.TX_INDEX
            ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
        ) AS Fksff_price,

        LAST_VALUE(t2_Czfq3.Czfq3_price IGNORE NULLS) OVER (
            ORDER BY t1.block, t2_Czfq3.TX_INDEX
            ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
        ) AS Czfq3_price,

        LAST_VALUE(t2_DB3sU.DB3sU_price IGNORE NULLS) OVER (
            ORDER BY t1.block, t2_DB3sU.TX_INDEX
            ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
        ) AS DB3sU_price,

        LAST_VALUE(t2_n9VhC.n9VhC_price IGNORE NULLS) OVER (
            ORDER BY t1.block, t2_n9VhC.TX_INDEX
            ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
        ) AS n9VhC_price,

        LAST_VALUE(t2_AvGeF.AvGeF_price IGNORE NULLS) OVER (
            ORDER BY t1.block, t2_AvGeF.TX_INDEX
            ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
        ) AS AvGeF_price,

        LAST_VALUE(t2_ucNos.ucNos_price IGNORE NULLS) OVER (
            ORDER BY t1.block, t2_ucNos.TX_INDEX
            ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
        ) AS ucNos_price,

        LAST_VALUE(t3_1.avg_volume_last_999_blocks IGNORE NULLS) OVER (
            ORDER BY t1.block
            ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
        ) AS avg_volume_last_999,

        -- exactly 1 block before
        b1.block_fee_sum AS sum_block_minus_1,
        b1.block_fee_avg AS avg_block_minus_1,

        -- exactly 2 blocks before
        b2.block_fee_sum AS sum_block_minus_2,
        b2.block_fee_avg AS avg_block_minus_2,

        -- exactly 3 blocks before
        b3.block_fee_sum AS sum_block_minus_3,
        b3.block_fee_avg AS avg_block_minus_3,

        -- exactly 4 blocks before
        b4.block_fee_sum AS sum_block_minus_4,
        b4.block_fee_avg AS avg_block_minus_4,

        -- exactly 5 blocks before
        b5.block_fee_sum AS sum_block_minus_5,
        b5.block_fee_avg AS avg_block_minus_5,

        -- exactly 6 blocks before
        b6.block_fee_sum AS sum_block_minus_6,
        b6.block_fee_avg AS avg_block_minus_6,

        -- exactly 7 blocks before
        b7.block_fee_sum AS sum_block_minus_7,
        b7.block_fee_avg AS avg_block_minus_7,

        -- exactly 8 blocks before
        b8.block_fee_sum AS sum_block_minus_8,
        b8.block_fee_avg AS avg_block_minus_8,

        -- exactly 9 blocks before
        b9.block_fee_sum AS sum_block_minus_9,
        b9.block_fee_avg AS avg_block_minus_9,

        -- exactly 10 blocks before
        b10.block_fee_sum AS sum_block_minus_10,
        b10.block_fee_avg AS avg_block_minus_10,

        t3_1.avg_volume_last_999_blocks

    FROM t1

    LEFT JOIN t2_Fksff ON t1.block = t2_Fksff.BLOCK_ID + 1
    LEFT JOIN t2_Czfq3 ON t1.block = t2_Czfq3.BLOCK_ID + 1
    LEFT JOIN t2_DB3sU ON t1.block = t2_DB3sU.BLOCK_ID + 1
    LEFT JOIN t2_n9VhC ON t1.block = t2_n9VhC.BLOCK_ID + 1
    LEFT JOIN t2_AvGeF ON t1.block = t2_AvGeF.BLOCK_ID + 1
    LEFT JOIN t2_ucNos ON t1.block = t2_ucNos.BLOCK_ID + 1

    LEFT JOIN t3   ON t1.block = t3.BLOCK_ID + 1
    LEFT JOIN t3_1 ON t1.block = t3_1.BLOCK_ID + 1

    LEFT JOIN t4 b1  ON b1.block  = t1.block - 1
    LEFT JOIN t4 b2  ON b2.block  = t1.block - 2
    LEFT JOIN t4 b3  ON b3.block  = t1.block - 3
    LEFT JOIN t4 b4  ON b4.block  = t1.block - 4
    LEFT JOIN t4 b5  ON b5.block  = t1.block - 5
    LEFT JOIN t4 b6  ON b6.block  = t1.block - 6
    LEFT JOIN t4 b7  ON b7.block  = t1.block - 7
    LEFT JOIN t4 b8  ON b8.block  = t1.block - 8
    LEFT JOIN t4 b9  ON b9.block  = t1.block - 9
    LEFT JOIN t4 b10 ON b10.block = t1.block - 10

    QUALIFY ROW_NUMBER() OVER (
        PARTITION BY t1.tx_id
        ORDER BY t2_Fksff.tx_index DESC
    ) = 1

    ORDER BY
        t1.block,
        t1.tx_id
),

/*
We now create 3 small intermediary dataframes for missing features needed.
First, we create a time dataframe that aggregates time by 30m blocks, which will be needed for a later merge in pandas with the seasonality dataframe
*/

time AS (
    SELECT
        MIN(BLOCK_TIMESTAMP),
        BLOCK_ID,
        MIN(TIME_SLICE(BLOCK_TIMESTAMP, 30, 'MINUTE')) AS block_ts_30m
    FROM SOLANA_ONCHAIN_CORE_DATA.CORE.FACT_TRANSACTIONS
    WHERE BLOCK_ID > $min_block_id
      AND BLOCK_ID < $max_block_id
    GROUP BY BLOCK_ID
    ORDER BY BLOCK_ID
    LIMIT $max_limit
),

/*
We create last_block, which basically counts the number of transactions in the current block (which will be the last block in the merge)
*/

last_block AS (
    SELECT
        COUNT(*) AS tx_count_in_block,
        MIN(block) AS block
    FROM t1
    WHERE block > $min_block_id
      AND block < $max_block_id
    GROUP BY block
    ORDER BY block
),

/*
We create last_total, which returns features for all transactions in a given block, not just the ones on our specific pool.
We thus return total transactions, block id (for the merge), total fees and units consumed
*/

last_total AS (
    SELECT
        COUNT(*) AS total_transactions,
        MIN(BLOCK_ID) AS BLOCK_ID,
        SUM(FEE) AS total_fee,
        SUM(UNITS_CONSUMED) AS units_consumed
    FROM SOLANA_ONCHAIN_CORE_DATA.CORE.FACT_TRANSACTIONS AS tr
    WHERE BLOCK_ID > $min_block_id
      AND BLOCK_ID < $max_block_id
    GROUP BY BLOCK_ID
    ORDER BY BLOCK_ID
)

/*
This is the final SELECT (the one we actually access in Python).
We only take values from the pre_final, time, last_block and last_total databases (with joins).
The only mathematical operation we do here is define the prices as their relative value compared to our target pool (in %).
*/


SELECT
    pre_final.block,
    priority_fee,
    fee_rank_desc,
    quantile_position,
    volume,
    avg_balance,
    Fksff_price,
    time.BLOCK_TS_30M,
    tx_count_in_block,
    last_total.total_transactions,
    last_total.total_fee,
    last_total.units_consumed,

    Czfq3_price,

    ((Czfq3_price - Fksff_price) / NULLIF(Fksff_price, 0)) * 100
        AS pct_diff_czfq3_fksff,

    ((Czfq3_price - DB3sU_price) / NULLIF(DB3sU_price, 0)) * 100
        AS pct_diff_czfq3_db3su,

    ((Czfq3_price - n9VhC_price) / NULLIF(n9VhC_price, 0)) * 100
        AS pct_diff_czfq3_n9vhc,

    ((Czfq3_price - AvGeF_price) / NULLIF(AvGeF_price, 0)) * 100
        AS pct_diff_czfq3_avgef,

    ((Czfq3_price - ucNos_price) / NULLIF(ucNos_price, 0)) * 100
        AS pct_diff_czfq3_ucnos,

    (
        (Czfq3_price - (
            (Fksff_price +
             DB3sU_price +
             n9VhC_price +
             AvGeF_price +
             ucNos_price) / 5
        ))
        /
        NULLIF(
            (Fksff_price +
             DB3sU_price +
             n9VhC_price +
             AvGeF_price +
             ucNos_price) / 5,
            0
        )
    ) * 100 AS pct_diff_czfq3_vs_avg,

    -- SUM history
    sum_block_minus_1,
    sum_block_minus_2,
    sum_block_minus_3,
    sum_block_minus_4,
    sum_block_minus_5,
    sum_block_minus_6,
    sum_block_minus_7,
    sum_block_minus_8,
    sum_block_minus_9,
    sum_block_minus_10,

    -- AVG history
    avg_block_minus_1,
    avg_block_minus_2,
    avg_block_minus_3,
    avg_block_minus_4,
    avg_block_minus_5,
    avg_block_minus_6,
    avg_block_minus_7,
    avg_block_minus_8,
    avg_block_minus_9,
    avg_block_minus_10,

    avg_volume_last_999

FROM pre_final
LEFT JOIN time
    ON pre_final.block = time.BLOCK_ID
LEFT JOIN last_block
    ON pre_final.block = last_block.block + 1
LEFT JOIN last_total
    ON pre_final.block = last_total.BLOCK_ID + 1

--Skip the first 50 transactions to handle potential missing values (in reality we would have used 1000, but since the test query is smaller, we only put 50 here)
WHERE pre_final.block > $min_block_id + 50

"""

In [5]:
seasonality = """

/*
We create temporary dataframe (just as before) with only priority fees, block and block_timestamp.
We group it by blocks (summing the priority fees per block)
We define the 30m time slices to to a grouping just after
*/

WITH t1 AS (
SELECT
            MIN(BLOCK_ID) AS block,
            MIN(BLOCK_TIMESTAMP),
            MIN(TIME_SLICE(BLOCK_TIMESTAMP, 30, 'MINUTE')) AS block_ts_30m,

            SUM(FEE - 5000) as priority_fee,

        FROM SOLANA_ONCHAIN_CORE_DATA.CORE.FACT_TRANSACTIONS AS tr,

             LATERAL FLATTEN(input => tr.instructions) AS inst,
             LATERAL FLATTEN(input => inst.value:accounts) AS acct
             
        WHERE
            BLOCK_ID > $min_season_block
            AND BLOCK_ID < $max_season_block
            AND SUCCEEDED = TRUE
            AND acct.value::string = 'Czfq3xZZDmsdGdUyrNLtRhGc47cXcZtLG4crryfu44zE'


        GROUP BY BLOCK_ID
        ORDER BY BLOCK_ID

    )

/*
Here, we select the dataframe we just created and group by 30m block, again summing the fees.
There probably would have been a way to do it in just one SELECT, but this seemed simpler.
*/

SELECT 
        MIN(block_ts_30m) as block_ts_30m,
        SUM(priority_fee) as priority_fee

FROM t1

GROUP BY block_ts_30m
ORDER BY block_ts_30m



"""

In [ ]:
# =============================================================================
# Loading both databases in Pandas using the Snowflake connector
# =============================================================================

main_df = pd.read_sql(main, conn)
seasonality_df = pd.read_sql(seasonality, conn)


# =============================================================================
# Export both databases to csv
# =============================================================================

main_df.to_csv("data_5k.csv")
seasonality_df.to_csv("seasonality_uncomplete.csv")
#Note: It's completely normal that there are only two rows in seasonality_uncomplete, it's because
#the block range used for this query is very small

C:\Users\killian\AppData\Local\Temp\ipykernel_36540\3849388530.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  main_df = pd.read_sql(main, conn)
C:\Users\killian\AppData\Local\Temp\ipykernel_36540\3849388530.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  seasonality_df = pd.read_sql(seasonality, conn)
